In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [37]:
df=pd.read_csv('data/loan_2014_18.csv')

In [39]:
df.head(10)

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,120122535,12000.0,12000.0,12000.0,36 months,7.97%,375.88,A,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1,119374887,32000.0,32000.0,32000.0,36 months,11.99%,1062.71,B,B5,...,Apr-2020,Jun-2020,Apr-2020,2.0,0.0,ACTIVE,123.08000,6189.66,1062.71,N
2,2,119321612,40000.0,40000.0,40000.0,60 months,15.05%,952.65,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,3,120122034,16000.0,16000.0,16000.0,36 months,7.97%,501.17,A,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,4,118659541,33000.0,33000.0,33000.0,36 months,7.21%,1022.12,A,A3,...,Feb-2020,Apr-2020,Feb-2020,3.0,21.0,DELINQUENT,177.96231,10197.78,59.68,N
5,5,119246018,7000.0,7000.0,7000.0,36 months,7.97%,219.26,A,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
6,6,119183129,10000.0,10000.0,10000.0,36 months,9.44%,320.05,B,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
7,7,120086024,8000.0,8000.0,8000.0,36 months,16.02%,281.34,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
8,8,119417303,12800.0,12800.0,12800.0,36 months,13.59%,434.93,C,C2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
9,9,120020939,8000.0,8000.0,8000.0,36 months,11.99%,265.68,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [40]:
df = df.drop(df.columns[0], axis=1)
df.shape

(2029952, 141)

### Identifying the target (y)
dependent variable : In our dataset the dependent variable js the "loan_status" column. First we check what types of categories it can take, then map it values as follows: </br> 'Fully Paid', 'Current', 'In Grace Period' - are map as '1' - positive </br> 'Late (16-30 days)', 'Late (31-120 days)', 'Charged Off', 'Default' - are map with '0' - negative

In [41]:
df['loan_status'].unique()

array(['Fully Paid', 'Current', 'Late (31-120 days)', 'Charged Off',
       'In Grace Period', 'Late (16-30 days)', 'Default'], dtype=object)

In [43]:
# Define the mapping
loan_status_mapping = {
    'Fully Paid': 1,
    'Current': 1,
    'In Grace Period': 1,
    'Late (16-30 days)': 0,
    'Late (31-120 days)': 0,
    'Charged Off': 0,
    'Default': 0
}

# Apply the mapping to the 'Loan_Status' column
df['loan_status_binary'] = df['loan_status'].map(loan_status_mapping)
df.drop('loan_status', axis=1, inplace=True)

df['loan_status_binary'].value_counts()

loan_status_binary
1    1704881
0     325071
Name: count, dtype: int64

In [44]:
missing_df = df.isnull().sum().to_frame(name='Missing')
missing_df['Total Count'] = len(df)
missing_df['Non-Missing'] = df.count()
missing_df['Missing Percentage'] = (missing_df['Missing'] / len(df)) * 100

In [55]:
missing_df = missing_df[(missing_df['Missing']>0) & (missing_df['Missing Percentage']>50)]
missing_df

,Missing,Total Count,Non-Missing,Missing Percentage
annual_inc_joint,1909242,2029952,120710,94.053554
dti_joint,1909246,2029952,120706,94.053751
verification_status_joint,1914222,2029952,115730,94.298880
revol_bal_joint,1921932,2029952,108020,94.678692
sec_app_fico_range_low,1921931,2029952,108021,94.678643
sec_app_fico_range_high,1921931,2029952,108021,94.678643
sec_app_earliest_cr_line,1921931,2029952,108021,94.678643
sec_app_inq_last_6mths,1921931,2029952,108021,94.678643
sec_app_mort_acc,1921931,2029952,108021,94.678643
sec_app_open_acc,1921931,2029952,108021,94.678643
